## Prepare Data

In [6]:
# โหลดข้อมูล, ไลบรารี
import pandas as pd
import geojson

# ข้อมูลดิบ
df_sheet1 = pd.read_excel("sheet1.xlsx")
df_sheet1 = df_sheet1.rename(columns={"ลำดับ": "ลำดับแปลง"})

# ข้อมูลน้ำบาดาลรัฐ
df_ckan_wells2 = pd.read_excel("Ckan_wells2.xlsx")
df_ckan_wells2 = df_ckan_wells2.sort_values(by='LATITUDE', ascending=False)
df_ckan_wells2.drop(df_ckan_wells2.tail(1).index,inplace=True) # drop last row เพราะเป็นค่าว่าง
df_ckan_wells2['Index'] = range(1, len(df_ckan_wells2) + 1)

## DataFrame to GeoJson

#### ข้อมูลแปลงไร่

In [26]:
# สร้าง GeoJSON features จาก DataFrame
features = []
for _, row in df_sheet1.iterrows():
    feature = geojson.Feature(
        geometry=geojson.Point((row['Longitude'], row['Latitude'])),
        properties={
            'ลำดับแปลง': row['ลำดับแปลง'],
            'ตัน/ไร่ประเมิน67/68': row['ตัน/ไร่ประเมิน67/68'],
            'ตัน/ไร่เกิดจริงปี66/67': row['ตัน/ไร่เกิดจริงปี66/67']
        }
    )
    features.append(feature)

# สร้าง GeoJSON FeatureCollection
feature_collection = geojson.FeatureCollection(features)

# แปลงเป็น string หรือบันทึกเป็นไฟล์ GeoJSON
geojson_str = geojson.dumps(feature_collection, ensure_ascii=False, indent=2)
with open('output.json', 'w', encoding='utf-8') as f:
    f.write(geojson_str)
print(geojson_str[:1000])  # แสดงเฉพาะ 1000 ตัวอักษรแรก

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          100.967404,
          15.626392
        ]
      },
      "properties": {
        "ลำดับแปลง": 1,
        "ตัน/ไร่ประเมิน67/68": 12.0,
        "ตัน/ไร่เกิดจริงปี66/67": "ไม่ได้ต่อสัญญาปี66/67"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          100.911557,
          15.422767
        ]
      },
      "properties": {
        "ลำดับแปลง": 2,
        "ตัน/ไร่ประเมิน67/68": 10.0,
        "ตัน/ไร่เกิดจริงปี66/67": "ไม่ได้ต่อสัญญาปี66/67"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          100.914545,
          15.692258
        ]
      },
      "properties": {
        "ลำดับแปลง": 3,
        "ตัน/ไร่ประเมิน67/68": 12.0,
        "ตัน/ไร่เกิดจริงปี66/67": "ไม่ได้ต่อสัญญาปี66/67"
      }
    

#### ข้อมูลบ่อบาดาลจากรัฐบาล

In [25]:
prov_list = ['ชัยภูมิ','นครราชสีมา','นครสวรรค์','พิจิตร', 'เพชรบูรณ์', 'ลพบุรี']
df_ckan_wells_filter = df_ckan_wells2[df_ckan_wells2['ชื่อจังหวัด'].isin(prov_list)]

# สร้าง GeoJSON features จาก DataFrame
features = []
for _, row in df_ckan_wells_filter.iterrows():
    feature = geojson.Feature(
        geometry=geojson.Point((row['LONGITUDE'], row['LATITUDE'])),
        properties={
            'หมายเลขบ่อน้ำบาดาล': row['หมายเลขบ่อน้ำบาดาล'],
            'ชื่อตำบล': row['ชื่อตำบล'],
            'ชื่ออำเภอ': row['ชื่ออำเภอ'],
            'ชื่อจังหวัด': row['ชื่อจังหวัด'],
            'ประเภทบ่อน้ำบาดาล': row['ประเภทบ่อน้ำบาดาล'],
            'ความลึกเจาะ': row['ความลึกเจาะ'],
            'ความลึกพัฒนา': row['ความลึกพัฒนา'],
            'ปริมาณน้ำ': row['ปริมาณน้ำ']
        }
    )
    features.append(feature)

# สร้าง GeoJSON FeatureCollection
feature_collection = geojson.FeatureCollection(features)

# แปลงเป็น string หรือบันทึกเป็นไฟล์ GeoJSON
geojson_str = geojson.dumps(feature_collection, ensure_ascii=False, indent=2)
with open('output_groundwater.json', 'w', encoding='utf-8') as f:
    f.write(geojson_str)
print(geojson_str[:1000])  # แสดงเฉพาะ 1000 ตัวอักษรแรก

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          101.391448,
          17.101176
        ]
      },
      "properties": {
        "หมายเลขบ่อน้ำบาดาล": "\t'PW15334'",
        "ชื่อตำบล": "ศิลา",
        "ชื่ออำเภอ": "หล่มเก่า",
        "ชื่อจังหวัด": "เพชรบูรณ์",
        "ประเภทบ่อน้ำบาดาล": "อุปโภค-บริโภค",
        "ความลึกเจาะ": 60.0,
        "ความลึกพัฒนา": 60.0,
        "ปริมาณน้ำ": 1.5
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          101.391448,
          17.101176
        ]
      },
      "properties": {
        "หมายเลขบ่อน้ำบาดาล": "\t'N599'",
        "ชื่อตำบล": "ศิลา",
        "ชื่ออำเภอ": "หล่มเก่า",
        "ชื่อจังหวัด": "เพชรบูรณ์",
        "ประเภทบ่อน้ำบาดาล": "อุปโภค-บริโภค",
        "ความลึกเจาะ": 36.0,
        "ความลึกพัฒนา": 36.0,
        "ปริมาณน้ำ": 3.18
      }
    },
    {
     